<a href="https://colab.research.google.com/github/Tatsuro0726/chemoinfomatics/blob/main/deepchem/Tutorial4_Fingerprint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Tutorial 4: Molecular Fingerprints
分子は多くの方法で表現することができます。このチュートリアルでは、「分子の指紋」と呼ばれる表現の一種を紹介します。これは非常にシンプルな表現で、小さな薬物のような分子によく機能することが多いです。

#### Setup

In [1]:
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3490  100  3490    0     0  22662      0 --:--:-- --:--:-- --:--:-- 22810


add /root/miniconda/lib/python3.6/site-packages to PYTHONPATH
python version: 3.6.9
fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
done
installing miniconda to /root/miniconda
done
installing rdkit, openmm, pdbfixer
added conda-forge to channels
added omnia to channels
done
conda packages installation finished!


# conda environments:
#
base                  *  /root/miniconda



In [2]:
!pip install --pre deepchem

     |████████████████████████████████| 532kB 4.5MB/s 


In [3]:
import deepchem as dc
dc.__version__

'2.5.0.dev'

#### Fingerprintとは？
Deeplearningでは、一般的に数値の配列を入力としている。分子を入力するには各分子を1つまたは複数の数値の配列として表現する必要がある。  
多くの予測モデルでは、入力のサイズを固定する必要がある。分子によって原子数が異なるため、分子を扱う上で難しい問題である。  

上記の問題の対処方法として、Fingerprintが提案されている。<br>Fingerprintは、固定長の配列で、各要素が分子の特徴を示している。２つの分子が似たようなFingerprintの場合、同じ特徴を多く含んでいることを示しており、化学的性質も似ている可能性が高いことが示される。  

DeepChemでは、Exteded Connectivity Fingerprint(ECFP)と呼ばれるフィンガープリントをサポートしている。<br>
ECFPアルゴリズムは、特性と結合に基づいて原子を分類することから始まる。それぞれの固有なパターンが特徴となる。例えば、「2つの水素と2つの重原子に結合した炭素原子」が特徴となり、この特徴を有する任意の分子に対して、Fingerprintの特定の要素に<b>"1"</b>が設定される。その次に、より大きな部分の近傍領域を見て、新しい特徴を反復的に識別する。他の2つの特定の特徴に結合した1つの特定の特徴が上位の特徴となり、その特徴を含むすべての分子に対応する要素が設定される。これを一定回数、多くの場合は2回繰り返す。

In [4]:
# ECPFで変換されたデータを見てみる
tasks, datasets, transformers = dc.molnet.load_tox21(featurizer='ECFP')
train, valid, test = datasets
print(train)

<DiskDataset X.shape: (6264, 1024), y.shape: (6264, 12), w.shape: (6264, 12), task_names: ['NR-AR' 'NR-AR-LBD' 'NR-AhR' ... 'SR-HSE' 'SR-MMP' 'SR-p53']>


y.shape(6264, 12)　⇒　マルチタスクデータセット
tox21には、毒性のサインを探すために12の異なるアッセイが使用された。

In [5]:
# 重み配列の確認
train.w

array([[1.0450224215246637, 1.036325992847732, 1.1250265336446614, ...,
        1.0557650327445922, 1.1746499631540162, 1.05288369419429],
       [1.0450224215246637, 1.036325992847732, 1.1250265336446614, ...,
        1.0557650327445922, 1.1746499631540162, 1.05288369419429],
       [1.0450224215246637, 1.036325992847732, 1.1250265336446614, ...,
        1.0557650327445922, 0.0, 1.05288369419429],
       ...,
       [1.0450224215246637, 0.0, 1.1250265336446614, ...,
        1.0557650327445922, 6.725738396624473, 1.05288369419429],
       [1.0450224215246637, 1.036325992847732, 1.1250265336446614, ...,
        1.0557650327445922, 6.725738396624473, 1.05288369419429],
       [1.0450224215246637, 1.036325992847732, 1.1250265336446614, ...,
        0.0, 1.1746499631540162, 1.05288369419429]], dtype=object)